In [1]:
import os
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
import jieba

In [2]:
# 定義主目錄
main_folder = '/home/b10173209/test.file/SS/class10(KFold)'

# 分類的清單
categories = ["anger","fear","joy","love","sadness","surprise"]

# 自定義 Tokenizer
def custom_tokenizer(text):
    tokens = jieba.lcut(text)
    return tokens

# 選擇要運行的 C 和 Kernel
C_value = 1
kernel = 'linear'

# 儲存每個折數的 DataFrame
all_validation_results = []


In [4]:

# 遍歷每個類別
for category in categories:
    category_folder = os.path.join(main_folder, category)

    # 遍歷每個子資料夾
    for i in range(5):
        subfolder = f'{category}_{i}'
        subfolder_path = os.path.join(category_folder, subfolder)

        # Debugging: Print folder path
        print(f'資料夾路徑: {subfolder_path}')

        # 檢查資料夾是否存在
        if not os.path.exists(subfolder_path):
            print(f'資料夾不存在: {subfolder_path}')
            continue

        # 讀取訓練集
        train_file = f'train_{category}_{i}.csv'
        train_df = pd.read_csv(os.path.join(subfolder_path, train_file))

        # 讀取測試集
        test_file = f'test_{category}_{i}.csv'
        test_df = pd.read_csv(os.path.join(subfolder_path, test_file))

        # 訓練集的標籤轉換為數字
        train_labels = train_df[category].apply(lambda x: 1 if x == 1 else 0).tolist()
        test_labels = test_df[category].apply(lambda x: 1 if x == 1 else 0).tolist()

        # 使用 StratifiedKFold 進行分層交叉驗證
        num_folds = 5
        skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

        # 遍歷每一折
        for fold, (train_index, val_index) in enumerate(skf.split(train_df, train_labels)):
            print(f"折數: {fold + 1}/{num_folds}")

            # 分割訓練和驗證集
            train_data, val_data = train_df.iloc[train_index], train_df.iloc[val_index]

            # 訓練集的標籤轉換為數字
            train_labels_fold = train_data[category].apply(lambda x: 1 if x == 1 else 0).tolist()

            # 驗證集的標籤轉換為數字
            val_labels_fold = val_data[category].apply(lambda x: 1 if x == 1 else 0).tolist()

            # 建立 TfidfVectorizer 將文本數據轉換為 TF-IDF 特徵
            vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer, max_features=None)
            X_train_fold = vectorizer.fit_transform(train_data['text'])
            X_val_fold = vectorizer.transform(val_data['text'])
            X_test = vectorizer.transform(test_df['text'])

            # 初始化支持向量機 (SVM) 模型
            svm_model = SVC(kernel=kernel, C=C_value, class_weight='balanced', random_state=42)

            # 訓練模型
            svm_model.fit(X_train_fold, train_labels_fold)

            # 在驗證集上進行預測
            predictions_val = svm_model.predict(X_val_fold)

            # 在測試集上進行預測
            predictions_test = svm_model.predict(X_test)

            # 將原始答案、預測結果、id 和捐款留言存入 DataFrame
            validation_results_df = pd.DataFrame({
                'Folder': [f'{subfolder}_{fold + 1}'] * len(val_data),
                'Fold': [fold + 1] * len(val_data),
                'id': val_data['id'].tolist(),
                'Labels': val_labels_fold,
                'Predictions': predictions_val,
                'text': val_data['text'].tolist()
            })

            # 將每個折數的結果存入列表
            all_validation_results.append(validation_results_df)

# 合併所有折數的結果
all_validation_results_df = pd.concat(all_validation_results)

# 存儲到 Excel 檔案
excel_path = f'/home/b10173209/test.file/SS/validation_results1_{kernel}_C_{C_value}.xlsx'
all_validation_results_df.to_excel(excel_path, index=False)


資料夾路徑: /home/b10173209/test.file/SS/class10(KFold)/anger/anger_0
折數: 1/5


/home/b10173209/venv/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


折數: 2/5


/home/b10173209/venv/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


折數: 3/5


...

/home/b10173209/venv/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


折數: 4/5


/home/b10173209/venv/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


折數: 5/5


/home/b10173209/venv/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
